In [1]:
import sys
import numpy as np
from pynq import Overlay
from pynq import allocate

# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
def load_xin_file(filename):
    data = np.zeros(shape=(104,200))
    with open(filename, 'r') as reader:
        file_data = reader.readlines()
        for idx, line in enumerate(file_data):
            line = line[:-1]
            line = line.split(" ")
            arr = np.asarray(line, dtype=np.float32)
            data[idx] = arr
    data = data.transpose()
    data = data[0]
    return data

def load_yin_file(filename):
    data = np.zeros(shape=(200,104))
    with open(filename, 'r') as reader:
        file_data = reader.readlines()
        for idx, line in enumerate(file_data):
            line = line.replace("{","")
            line = line.replace("}","")
            line = line[:-2]
            line = line.split(",")
            if len(line) == 1:
                continue
            arr = np.asarray(line, dtype=np.float32)
            data[idx] = arr
    return data

def load_actual_file(filename):
    data = np.zeros(shape=(104,200))
    with open(filename, 'r') as reader:
        file_data = reader.readlines()
        for idx, line in enumerate(file_data):
            line = line[:-1]
            line = line.split(" ")
            arr = np.asarray(line, dtype=np.float32)
            data[idx] = arr
            
    data = data.transpose()
    return data

def run_ip(yin, xin):
    for idx in range(2*104):
        if idx<104:
            in_buffer[idx] = yin[idx]
        else:
            in_buffer[idx] = xin[idx-104]
    
    leastsq.write(0x00, 0x1)
    
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    return out_buffer

In [3]:
box_layout = widgets.Layout(display='flex',
                flex_flow='column',
                align_items='center',
                width='100%')

btn_load_IP = widgets.Button(description='Load the IP')
btn_run_ip = widgets.Button(description='Run IP')
btn_plot = widgets.Button(description='Plot')
out = widgets.Output()

data = np.zeros(shape=(104,200))
def btn_load_IP_on_click(_):
     # "linking function with output"
    with out:
        # what happens when we press the button
        clear_output()
        global overlay
        global dma
        global leastsq
        global in_buffer
        global out_buffer
        
        overlay = Overlay('./../misc/ls_ip.bit')
        dma = overlay.axi_dma_0
        leastsq = overlay.leastSquare_0
        
        in_buffer = allocate(shape=(2*104,), dtype=np.float32)
        out_buffer = allocate(shape=(104,), dtype=np.float32)
        print("IP loaded!")

def btn_run_ip_on_click(_):
    # "linking function with output"
    with out:
    # what happens when we press the button
        clear_output()
        print("Running IP...\n") 
        
        global N_SNR
        global N_CH
        global IP_output
        global Err_DNN_hw
        global Phf_hw
        global Err_DNN_normalized_hw
        
        N_SNR = 7
        N_CH = 200

        IP_output = np.zeros([N_SNR, N_CH, 104])
        Err_DNN_hw = np.zeros([N_SNR,])
        Phf_hw = np.zeros([N_SNR,])

        for n_snr in range(N_SNR):
            print(f"Signal: {n_snr+1}")

            xin = load_xin_file("./../data/XinC_1.txt")
            yin = load_yin_file(f"./../data/Yinc_m{n_snr+1}m.dat")
            act = load_actual_file(f"./../data/actual{n_snr+1}.txt")[0:N_CH]

            for n_ch in range(N_CH):
                print(f"  Channel: {n_ch+1}\r", end="")

                op = run_ip(yin[n_ch], xin)
                op = op[:104]
                IP_output[n_snr,n_ch,:] = op

            # 2-norm (largest sing. value)
            Err_DNN_hw[n_snr] += np.linalg.norm(act - IP_output[n_snr], 2)**2
            Phf_hw[n_snr] +=  np.linalg.norm(act, 2)**2
            print("\n")
        Err_DNN_hw /=  104
        Phf_hw /= 104
        Err_DNN_normalized_hw = Err_DNN_hw / Phf_hw
        print("IP run finished!") 

def btn_plot_on_click(_):
    # "linking function with output"
    with out:
    # what happens when we press the button
        clear_output()
        plt.semilogy(np.arange(0, N_SNR*5, 5), Err_DNN_normalized_hw)
        plt.xlabel("Preamble SNR")
        plt.ylabel("Average Error per subcarrier")
        plt.show()
        
# linking button and function together using a button's method
btn_load_IP.on_click(btn_load_IP_on_click)
btn_run_ip.on_click(btn_run_ip_on_click)
btn_plot.on_click(btn_plot_on_click)

# displaying button and its output together
widgets.HBox([btn_load_IP, btn_run_ip, btn_plot, out], layout=box_layout)

In [4]:
Err_DNN_normalized_hw

array([1.17738927e+00, 3.79963109e-01, 1.16320961e-01, 3.58048640e-02,
       1.07467376e-02, 3.85467260e-03, 9.35095608e-04])

In [5]:
IP_output[0]

array([[-1.03812897, -0.89598602,  0.1085    , ...,  0.70032901,
         0.113018  , -0.30190101],
       [ 0.157701  ,  0.63607299,  1.21284997, ..., -0.91760498,
        -0.66222203,  0.200697  ],
       [ 0.74327499,  1.10473502, -0.63569999, ..., -0.98781198,
        -1.51866806, -1.83411503],
       ...,
       [ 0.82737899,  0.90943497, -1.100577  , ...,  0.51791501,
         0.415315  ,  0.116634  ],
       [-0.27631101, -0.171382  ,  0.34815499, ..., -1.04417801,
        -1.10378003, -0.83997703],
       [ 0.88639802,  0.54068798, -0.97302598, ...,  0.129384  ,
         0.25719199,  0.016987  ]])